In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

sns.set_theme()


def get_best_known_score(instance: str, problem: str):
    file = f"instances/best_scores_{problem}.txt"
    with open(file, "r", encoding="utf8") as file:
        for line in file.readlines():
            instance_, score, optimal = line[:-1].split(" ")
            if instance_ == instance:
                return int(score), optimal == "*"
    raise Exception(f"instance {instance} not found in {file}")


lss = [
    "afisa_original",
    "tabu_weight",
    "redls",
    "ilsts",
    # "random",
    # "deleter",
    # "roulette_wheel",
    # "pursuit",
    # "ucb",
    # "neural_net",
    # "depth",
    # "depth_fit",
    # "fit",
    # "level",
    # "chance"
]

title = {
    "afisa_original": "AFISA",
    "tabu_weight": "TabuWeight",
    "redls": "RedLS",
    "ilsts": "ILSTS",
    "random": "Random",
    "deleter": "deleter",
    "roulette_wheel": "Roulette wheel",
    "pursuit": "Pursuit",
    "ucb": "UCB",
    "neural_net": "Neural Net",
    "depth": "depth",
    "depth_fit": "depth_fit",
    "fit": "fit",
    "level": "level",
    "chance": "chance",
}
problem = "wvcp"

repertory = "outputs/mcts_ls_all/"
repertory = "outputs/all_methods_all_instances/"

nb_hours = 1
nb_rand_seed = 20

time_limit = 3600 * nb_hours

colors = sns.color_palette("husl", len(lss))


# Choose the set of instances
instances_set = ("pxx", "pxx")
instances_set = ("rxx", "rxx")
instances_set = ("DIMACS_non_optimal", "dimacs_no")
instances_set = ("DIMACS_optimal", "dimacs_o")
instances_set = ("../instances_coeff", "instances_coeff")
instances_set = ("instance_list_wvcp", "all")
instances_set = ("../instances_hard_wvcp", "hard_wvcp")

output_file = f"xlsx_files/mcts_ls_before_{instances_set[1]}.xlsx"

with open(f"instances/{instances_set[0]}.txt", "r", encoding="utf8") as file:
    instances = [i[:-1] for i in file.readlines()]

for instance in instances:
    best_score, optimality = get_best_known_score(instance, problem)
    fig, ax = plt.subplots(figsize=(10, 10))

    # fig, ax = plt.subplots(figsize=(5.5, 5.5))
    ax.hlines(
        y=best_score,
        xmin=0,
        xmax=time_limit,
        color="black" if optimality else "gray",
        linestyle="-",
        label="optimal" if optimality else "best known",
    )
    ax.text(
        time_limit,
        best_score,
        "{:.0f}".format(best_score),
        color="black" if optimality else "gray",
        ha="left",
        va="center",
    )

    for i, p in enumerate(lss):
        times_x = []
        score_y = []
        for rand_seed in range(nb_rand_seed):
            file = f"{repertory}/{p}/{instance}_{rand_seed}.csv"
            data = pd.read_csv(file, comment="#")[
                ["score", "turn", "time", "nb_colors"]
            ]
            data.at[len(data) - 1, "time"] = time_limit
            times_x.append(data.time.to_list())
            score_y.append(data.score.to_list())
            # ax.plot(data.time, data.score, label=f"{p}", color=colors[i])
        max_time = max(times[-1] for times in times_x)
        x_axis = [i for i in range(max_time)]
        # interpolation of the score for each rand seed
        score_interpolations = [
            np.interp(x_axis, times_x[i], score_y[i]) for i in range(nb_rand_seed)
        ]
        # mean of the interpolations of the scores
        y_axis = np.mean(score_interpolations, axis=0)

        ax.plot(x_axis, y_axis, label=f"{p}", color=colors[i])

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.xlabel("time(m)")
    plt.xticks([i * 3600 for i in range(nb_hours + 1)], list(range(nb_hours + 1)))
    ticks_x = ticker.FuncFormatter(lambda x, pos: f"{x / 3600:0.0f}")
    ax.xaxis.set_major_formatter(ticks_x)
    plt.ylabel("score")
    plt.xlim(left=0, right=nb_hours * 3600)
    plt.grid(True, which="both", linestyle="--")
    plt.title(f"{instance}")
    # plt.savefig(f"coeff_{instance}.png", bbox_inches="tight")
    plt.show()


In [ ]:
# "Accent", "Accent_r",
# "Blues", "Blues_r", "BrBG", "BrBG_r", "BuGn", "BuGn_r", "BuPu", "BuPu_r",
# "CMRmap", "CMRmap_r",
# "Dark2", "Dark2_r",
# "GnBu", "GnBu_r", "Greens", "Greens_r", "Greys", "Greys_r",
# "OrRd", "OrRd_r", "Oranges", "Oranges_r",
# "PRGn", "PRGn_r", "Paired", "Paired_r", "Pastel1", "Pastel1_r", "Pastel2", "Pastel2_r", "PiYG", "PiYG_r", "PuBu", "PuBuGn", "PuBuGn_r", "PuBu_r", "PuOr", "PuOr_r", "PuRd", "PuRd_r", "Purples", "Purples_r",
# "RdBu", "RdBu_r", "RdGy", "RdGy_r", "RdPu", "RdPu_r", "RdYlBu", "RdYlBu_r", "RdYlGn", "RdYlGn_r", "Reds", "Reds_r",
# "Set1", "Set1_r", "Set2", "Set2_r", "Set3", "Set3_r", "Spectral", "Spectral_r",
# "Wistia", "Wistia_r",
# "YlGn", "YlGnBu", "YlGnBu_r", "YlGn_r", "YlOrBr", "YlOrBr_r", "YlOrRd", "YlOrRd_r",
# "afmhot", "afmhot_r", "autumn", "autumn_r",
# "binary", "binary_r", "bone", "bone_r", "brg", "brg_r", "bwr", "bwr_r",
# "cividis", "cividis_r", "cool", "cool_r", "coolwarm", "coolwarm_r", "copper", "copper_r", "crest", "crest_r", "cubehelix", "cubehelix_r",
# "flag", "flag_r", "flare", "flare_r",
# "gist_earth", "gist_earth_r", "gist_gray", "gist_gray_r", "gist_heat", "gist_heat_r", "gist_ncar", "gist_ncar_r", "gist_rainbow", "gist_rainbow_r", "gist_stern", "gist_stern_r", "gist_yarg", "gist_yarg_r", "gnuplot", "gnuplot2", "gnuplot2_r", "gnuplot_r", "gray", "gray_r",
# "hot", "hot_r", "hsv", "hsv_r",
# "icefire", "icefire_r", "inferno", "inferno_r",
# "jet", "jet_r",
# "magma", "magma_r", "mako", "mako_r",
# "nipy_spectral", "nipy_spectral_r",
# "ocean", "ocean_r", "pink", "pink_r", "plasma", "plasma_r", "prism", "prism_r",
# "rainbow", "rainbow_r", "rocket", "rocket_r",
# "seismic", "seismic_r", "spring", "spring_r", "summer", "summer_r",
# "tab10", "tab10_r", "tab20", "tab20_r", "tab20b", "tab20b_r", "tab20c", "tab20c_r", "terrain", "terrain_r", "turbo", "turbo_r", "twilight", "twilight_r", "twilight_shifted", "twilight_shifted_r",
# "viridis", "viridis_r", "vlag", "vlag_r",
# "winter", "winter_r"
